In [1]:
import os
from scipy.io import loadmat
import pandas as pd
import kagglehub
from torch.utils.data import Dataset
from PIL import Image
from torchvision.transforms import transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import numpy as np
import matplotlib.pyplot as plt

## General rules
1. update group when working and what sections
2. Try to be comprehensive as you write! Leave short markdown descriptions for the average reader to understand what you're doing

## Overview
### [FILL IN WITH PROJECT DESCRIPTION]

## Data

In [2]:
# Download latest version
path = kagglehub.dataset_download("eduardo4jesus/stanford-cars-dataset")

print("Path to dataset files:", path)

Path to dataset files: /Users/bassamhajjawi/.cache/kagglehub/datasets/eduardo4jesus/stanford-cars-dataset/versions/1


In [3]:
# Explore directories/files
print("Path:", path)
print("Folders/files inside the dataset:")
dir = os.listdir(path)
print(dir)
devpath = path + '/car_devkit/devkit'
print('Files inside devkit')
print(os.listdir(devpath))
metapath = devpath + "/cars_meta.mat"

Path: /Users/bassamhajjawi/.cache/kagglehub/datasets/eduardo4jesus/stanford-cars-dataset/versions/1
Folders/files inside the dataset:
['cars_train', 'cars_test', 'car_devkit']
Files inside devkit
['cars_test_annos.mat', 'eval_train.m', 'cars_meta.mat', 'README.txt', 'cars_train_annos.mat', 'train_perfect_preds.txt']


In [4]:
meta = loadmat(metapath)

train_dir = os.path.join(path, 'cars_train' )
test_dir = os.path.join(path, 'cars_test')
class_names = [c[0] for c in meta["class_names"][0]]
class_index = range(1, len(class_names) + 1) # classes are labeled w/ 1 based labeling

classes = pd.DataFrame( {
    "label": class_index,
    "class_names" : class_names
})
classes

,label,class_names
0,1,AM General Hummer SUV 2000
1,2,Acura RL Sedan 2012
2,3,Acura TL Sedan 2012
3,4,Acura TL Type-S 2008
4,5,Acura TSX Sedan 2012
...,...,...
191,192,Volkswagen Beetle Hatchback 2012
192,193,Volvo C30 Hatchback 2012
193,194,Volvo 240 Sedan 1993
194,195,Volvo XC90 SUV 2007


In [5]:
train_ann = loadmat(os.path.join(devpath, 'cars_train_annos.mat'))
train_ann = train_ann["annotations"][0] # index 4 is label for each train image
test_ann = loadmat(os.path.join(devpath, 'cars_test_annos.mat'))
test_ann = test_ann["annotations"][0] # no test labels, will split up train_ann to train/test
train_labels = [i[4][0][0] for i in train_ann]
file_names = [i[5][0] for i in train_ann]


data_dict = {
    "filen" : file_names,
    "train_labels" : train_labels
}

data = pd.DataFrame(data_dict)

data["class_name"] = data["train_labels"].map(
    dict(zip(classes["label"], classes["class_names"]))
)
data

,filen,train_labels,class_name
0,00001.jpg,14,Audi TTS Coupe 2012
1,00002.jpg,3,Acura TL Sedan 2012
2,00003.jpg,91,Dodge Dakota Club Cab 2007
3,00004.jpg,134,Hyundai Sonata Hybrid Sedan 2012
4,00005.jpg,106,Ford F-450 Super Duty Crew Cab 2012
...,...,...,...
8139,08140.jpg,78,Chrysler Town and Country Minivan 2012
8140,08141.jpg,196,smart fortwo Convertible 2012
8141,08142.jpg,163,Mercedes-Benz SL-Class Coupe 2009
8142,08143.jpg,112,Ford GT Coupe 2006


## Body types work:

In [6]:
body_types = [
    "Coupe", "Sedan", "SUV", "Hatchback",
    "Convertible", "Minivan", "Wagon", "Crossover", "Van", "Cab"
]

def find_body_type(name):
    name_lower = name.lower()
    
    for body in body_types:
        if body.lower() in name_lower:
            return body
    #coupes
    if any(x in name_lower for x in ["corvette", "camaro", "mustang", "challenger", "370z", "350z", "supra", "xk", "xkr", "integra", "gallardo"]):
        return "Coupe"

    # Sedans
    if any(x in name_lower for x in ["charger", "chrysler 300", "cobalt", "impala", "malibu", "accord", "civic", "corolla", 
                                     "jetta", "regal", "tl type-s"]):
        return "Sedan"

    # SUVs 
    if any(x in name_lower for x in ["grand cherokee", "cherokee", "durango", "rav4", "cr-v", "crv", "rogue", "highlander", "pilot", "tahoe", 
                                     "explorer", "escape", "equinox","trailblazer"]):
        return "SUV"

    #hatchbacks
    if any(x in name_lower for x in ["golf", "fit", "hhr", "impreza hatch", "mazda3 hatch", "sportwagen","fiat 500"]):
        return "Hatchback"

    # Trucks
    if any(x in name_lower for x in ["f-150", "f150", "f-250", "ram", "silverado", "sierra", "tacoma", "tundra", "ranger", "colorado"]):
        return "Truck"

    # Vans / Minivans
    if any(x in name_lower for x in ["caravan", "odyssey", "sienna", "transit", "express van", "sprinter"]):
        return "Van"

    # other
    if "srt" in name_lower or "ss" in name_lower or "hellcat" in name_lower or "z06" in name_lower or "zl1" in name_lower:
        if any(x in name_lower for x in ["charger", "chrysler 300", "cts"]):
            return "Sedan"
        if any(x in name_lower for x in ["corvette", "camaro", "challenger", "mustang"]):
            return "Coupe"

    if "cab" in name_lower:
        return "Cab"
    return "Unknown"

data["body type"] = data["class_name"].apply(find_body_type)
data.loc[data["body type"] == "Cab", "body type"] = "Truck"
data.loc[data["body type"] == "Minivan", "body type"] = "Van"

In [7]:
data

,filen,train_labels,class_name,body type
0,00001.jpg,14,Audi TTS Coupe 2012,Coupe
1,00002.jpg,3,Acura TL Sedan 2012,Sedan
2,00003.jpg,91,Dodge Dakota Club Cab 2007,Truck
3,00004.jpg,134,Hyundai Sonata Hybrid Sedan 2012,Sedan
4,00005.jpg,106,Ford F-450 Super Duty Crew Cab 2012,Truck
...,...,...,...,...
8139,08140.jpg,78,Chrysler Town and Country Minivan 2012,Van
8140,08141.jpg,196,smart fortwo Convertible 2012,Convertible
8141,08142.jpg,163,Mercedes-Benz SL-Class Coupe 2009,Coupe
8142,08143.jpg,112,Ford GT Coupe 2006,Coupe


In [8]:
# Show that all data is labeled
unknowns = data[data["body type"] == "Unknown"]
print(unknowns)


Empty DataFrame
Columns: [filen, train_labels, class_name, body type]
Index: []


Body types end

In [9]:
# Need to construct custom Dataset Class to later use for DataLoader for models
class CarsDataset(Dataset):
    def __init__(self, df, images_dir, transform=None, supervised=True, use_body_type=False, body_type_to_idx=None):
        self.df = df
        self.images_dir = images_dir
        self.transform = transform
        self.supervised = supervised
        self.use_body_type = use_body_type
        self.body_type_to_idx = body_type_to_idx
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, ind):
        row = self.df.iloc[ind]
        img_path = ''
        if 'train' in self.images_dir:
            img_path = os.path.join(self.images_dir, 'cars_train') # folder is nested /cars_train/cars_train
        else:
            img_path = os.path.join(self.images_dir, 'cars_test')

        img_path = os.path.join(img_path, row["filen"]) # path to specific image
        img = Image.open(img_path).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)

        if self.supervised:
            if self.use_body_type and self.body_type_to_idx is not None:
                # Use body type label
                body_type = row["body type"]
                label = self.body_type_to_idx[body_type]
            else:
                # Use original car model label
                label = row["train_labels"]
            return img, label
        else: # unsupervised; no image needed
            return img 

## Supervised Learning
### [ QUICK OVERVIEW OF SECTION ]

In [10]:
# Define transformations on images
res_transforms = transforms.Compose(
    [
        transforms.Resize((224,224)), #resnets expect 224,224
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # ImageNet normalization
    ]
)

In [11]:
# Split dataset into train/val/test (70/15/15)
train_df, valtest_df = train_test_split(data, test_size=0.3, stratify=data["train_labels"])
val_df, test_df = train_test_split(valtest_df, test_size=0.5, stratify=valtest_df["train_labels"])

# construct CarsDatasets()
train_data = CarsDataset(train_df, train_dir, transform=res_transforms, supervised=True)
val_data = CarsDataset(val_df, train_dir, transform=res_transforms, supervised=True)
test_data = CarsDataset(test_df, train_dir, transform=res_transforms, supervised=True)

In [12]:
# Fix: Find the correct path where images actually are (handles nested directory structure)
# This ensures train_dir points to the directory that actually contains the .jpg files
actual_train_dir = train_dir
if os.path.exists(train_dir):
    contents = os.listdir(train_dir)
    # Check if there's a nested cars_train directory
    if 'cars_train' in contents and os.path.isdir(os.path.join(train_dir, 'cars_train')):
        actual_train_dir = os.path.join(train_dir, 'cars_train')
        print(f"Found nested directory! Using: {actual_train_dir}")
    # Check if there are .jpg files directly
    jpg_files = [f for f in contents if f.endswith('.jpg')]
    if jpg_files:
        print(f"Found {len(jpg_files)} .jpg files directly in train_dir")
        actual_train_dir = train_dir
    else:
        # Search recursively for .jpg files
        for root, dirs, files in os.walk(train_dir):
            jpg_count = len([f for f in files if f.endswith('.jpg')])
            if jpg_count > 0:
                actual_train_dir = root
                print(f"Found {jpg_count} .jpg files in: {actual_train_dir}")
                break

# Update train_dir to the correct path (Body Type Setup cell will use this)
train_dir = actual_train_dir
print(f"\ntrain_dir updated to: {train_dir}")
print("Note: Datasets will be recreated in the Body Type Setup cell with the correct path")


Found nested directory! Using: /Users/bassamhajjawi/.cache/kagglehub/datasets/eduardo4jesus/stanford-cars-dataset/versions/1/cars_train/cars_train
Found 8144 .jpg files in: /Users/bassamhajjawi/.cache/kagglehub/datasets/eduardo4jesus/stanford-cars-dataset/versions/1/cars_train/cars_train

train_dir updated to: /Users/bassamhajjawi/.cache/kagglehub/datasets/eduardo4jesus/stanford-cars-dataset/versions/1/cars_train/cars_train
Note: Datasets will be recreated in the Body Type Setup cell with the correct path


#### [TODO]
- construct DataLoaders with Datasets
- import models: resnet18, resnet50, densenet 121, etc etc.
- define model (layers, inputs, optimizer, loss)
- train
- evaluate
- repeat?


In [13]:
# Setup for Body Type Prediction (instead of 196 car models)
# Get unique body types and create mapping
unique_body_types = sorted(data["body type"].unique())
body_type_to_idx = {body_type: idx for idx, body_type in enumerate(unique_body_types)}
idx_to_body_type = {idx: body_type for body_type, idx in body_type_to_idx.items()}

num_body_types = len(unique_body_types)
print(f"Body types to predict: {unique_body_types}")
print(f"Number of body type classes: {num_body_types}")
print(f"Body type mapping: {body_type_to_idx}")

# Recreate datasets with body type labels
train_data = CarsDataset(train_df, train_dir, transform=res_transforms, supervised=True, 
                        use_body_type=True, body_type_to_idx=body_type_to_idx)
val_data = CarsDataset(val_df, train_dir, transform=res_transforms, supervised=True,
                      use_body_type=True, body_type_to_idx=body_type_to_idx)
test_data = CarsDataset(test_df, train_dir, transform=res_transforms, supervised=True,
                       use_body_type=True, body_type_to_idx=body_type_to_idx)

print(f"\nDatasets recreated for body type prediction")


Body types to predict: ['Convertible', 'Coupe', 'Hatchback', 'SUV', 'Sedan', 'Truck', 'Van', 'Wagon']
Number of body type classes: 8
Body type mapping: {'Convertible': 0, 'Coupe': 1, 'Hatchback': 2, 'SUV': 3, 'Sedan': 4, 'Truck': 5, 'Van': 6, 'Wagon': 7}

Datasets recreated for body type prediction


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.models import resnet50, ResNet50_Weights
import numpy as np


In [15]:
# Step 1: Create DataLoaders for batching data
batch_size = 32

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=0)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")
print(f"Test batches: {len(test_loader)}")


Train batches: 179
Val batches: 39
Test batches: 39


In [16]:
# Step 2: Set up ResNet50 model for BODY TYPE prediction
# Using number of body type classes (not 196 car models)
num_classes = num_body_types  # This was set in the body type setup cell

# Load pretrained ResNet50 with ImageNet weights
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)

# Replace the final fully connected layer for body type classes
# ResNet50's fc layer expects 2048 input features (from avgpool)
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Move model to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"Model loaded on: {device}")
print(f"Number of classes (body types): {num_classes}")
print(f"Predicting: {unique_body_types}")


Model loaded on: cpu
Number of classes (body types): 8
Predicting: ['Convertible', 'Coupe', 'Hatchback', 'SUV', 'Sedan', 'Truck', 'Van', 'Wagon']


In [17]:
# Step 3: Define loss function, optimizer, and learning rate scheduler
criterion = nn.CrossEntropyLoss()  # Standard classification loss
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate 0.001

# Learning rate scheduler: reduces LR by factor of 0.1 every 7 epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

print("Loss function: CrossEntropyLoss")
print("Optimizer: Adam (lr=0.001)")
print("Scheduler: StepLR (step_size=7, gamma=0.1)")


Loss function: CrossEntropyLoss
Optimizer: Adam (lr=0.001)
Scheduler: StepLR (step_size=7, gamma=0.1)


In [18]:
# Step 4: Training function
def train_epoch(model, train_loader, criterion, optimizer, device):
    """
    Train the model for one epoch
    
    Returns:
        epoch_loss: average loss for the epoch
        epoch_acc: accuracy percentage for the epoch
    """
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        # Move data to device (GPU or CPU)
        images = images.to(device)
        labels = labels.to(device)  # Body type labels are already 0-based (0 to num_body_types-1)
        
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        
        # Forward pass: compute predictions
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass: compute gradients
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        # Track statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)  # Get predicted class
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc


In [19]:
# Step 5: Validation function
def validate(model, val_loader, criterion, device):
    """
    Validate the model on validation set
    
    Returns:
        epoch_loss: average loss for the epoch
        epoch_acc: accuracy percentage for the epoch
    """
    model.eval()  # Set model to evaluation mode (disables dropout, etc.)
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():  # Disable gradient computation for efficiency
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)  # Body type labels are already 0-based
            
            # Forward pass only (no backprop)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Track statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100 * correct / total
    return epoch_loss, epoch_acc


In [21]:
# Step 6: Training loop
num_epochs = 10  # Number of complete passes through the training data
best_val_acc = 0.0  # Track best validation accuracy

# Lists to store training history
train_losses = []
train_accs = []
val_losses = []
val_accs = []

for epoch in range(num_epochs):
    print(f'\nEpoch {epoch+1}/{num_epochs}')
    print('-' * 20)
    
    # Train for one epoch
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # Validate
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    # Update learning rate
    scheduler.step()
    
    # Print results
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
    
    # Save best model based on validation accuracy
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_resnet50_model.pth')
        print(f'✓ New best model saved! (Val Acc: {val_acc:.2f}%)')

print(f'\nTraining complete! Best validation accuracy: {best_val_acc:.2f}%')



Epoch 1/10
--------------------


KeyboardInterrupt: 

In [ ]:
print("Evaluating on test set...")
test_loss, test_acc = validate(model, test_loader, criterion, device)
print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')


Evaluating on test set...
Test Loss: 0.6363, Test Acc: 83.06%


## Unsupervised Learning
### [QUICK OVERVIEW OF SECTION]

Let's define some helper functions that we can use later on to help us with our data. These were used in HW4 from COGS 118B FA25.

In [90]:
def norm(mat):
    return normalize(mat, norm='l2', axis=0) #normalize columns of matrix mat to unit length 1

def viewimage(column):
    normalized = (column - min(column)) / (max(column) - min(column))

    return plt.imshow(normalized.reshape([60,60,3])) #reshape into RGB dimensions

def eigsort(V, eigvals):
    # [Vsort,Dsort] = eigsort(V, eigvals)
    #
    # Sorts a matrix eigenvectors and a array of eigenvalues in order 
    # of eigenvalue size, largest eigenvalue first and smallest eigenvalue
    # last.
    #
    # Example usage:
    # di, V = np.linarg.eig(L)
    # Vnew, Dnew = eigsort(V, di)
    #
    # Tim Marks 2002
    
    # Sort the eigenvalues from largest to smallest. Store the sorted
    # eigenvalues in the column vector lambd.
    lohival = np.sort(eigvals)
    lohiindex = np.argsort(eigvals)
    lambd = np.flip(lohival)
    index = np.flip(lohiindex)
    Dsort = np.diag(lambd)
    
    # Sort eigenvectors to correspond to the ordered eigenvalues. Store sorted
    # eigenvectors as columns of the matrix vsort.
    M = np.size(lambd)
    Vsort = np.zeros((M, M))
    for i in range(M):
        Vsort[:,i] = V[:,index[i]]
    return Vsort, Dsort

First we want to initialize our CarsDataaset using our "supervised" parameter we defined in the Data section. This lets us ignore the labels of each image, since we don't need them right now. Let's look at the image size of the first few images to see whether they're all the same.